In [11]:
# CELDA 1: Imports
# Todo lo necesario
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle # Para guardar el modelo y listas de columnas


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [12]:
# CELDA 2: Carga de Datos de Entrenamiento
# Reemplaza con tu ruta correcta
path = r"C:\Users\dalon\SynologyDrive\Uni\2024-2025\HaCoBu\HaCoBu\navegacion_maritima\datos_etiquetados" 

lista_df=[]
archivos_carpeta=[]
df_datos = pd.DataFrame() # Inicializar por si no se cargan archivos

try:
    archivos_carpeta = os.listdir(path)
    if not archivos_carpeta:
        print(f"Advertencia: No se encontraron archivos en la carpeta '{path}'")
    else:
        print(f"Archivos/carpetas encontrados en '{path}': {len(archivos_carpeta)} items.")
except FileNotFoundError:
    print(f"Error: La carpeta '{path}' no fue encontrada. Verifica la ruta.")
except Exception as e:
    print(f"Ocurrió un error al acceder a la carpeta: {e}")
    
contador_cargados = 0
if archivos_carpeta:
    for archivo in archivos_carpeta:
        if archivo.lower().endswith('.csv'):
            ruta_archivo = os.path.join(path, archivo)
            try:
                df_temporal = pd.read_csv(ruta_archivo)
                lista_df.append(df_temporal)
                contador_cargados += 1
            except Exception as e:
                print (f"Error al cargar '{archivo}': {e}")
            
if lista_df:
    df_datos = pd.concat(lista_df, ignore_index=True)
    print(f"\nSe combinaron {contador_cargados} archivos CSV de entrenamiento.")
    print(f"df_datos (entrenamiento) cargado con {df_datos.shape[0]} filas y {df_datos.shape[1]} columnas.")
else:
    if archivos_carpeta:
        print(f"\nNo se cargó ningún archivo CSV de la carpeta '{path}'.")
    print("El DataFrame df_datos está vacío.")

Archivos/carpetas encontrados en 'C:\Users\dalon\SynologyDrive\Uni\2024-2025\HaCoBu\HaCoBu\navegacion_maritima\datos_etiquetados': 100 items.

Se combinaron 100 archivos CSV de entrenamiento.
df_datos (entrenamiento) cargado con 1150 filas y 15 columnas.


In [13]:
# CELDA 3: Visualización Opcional (Entrenamiento)
if not df_datos.empty:
    print("\nPrimeras filas de df_datos (entrenamiento original):")
    display(df_datos.head())
else:
    print("\nEl DataFrame df_datos (entrenamiento) está vacío.")


Primeras filas de df_datos (entrenamiento original):


Ejercicio  Situacion_n  DistanciaInicioRiiesgo  DCPA_yds   Situacion  \
0          1            1                 6803.80   1254.56  Alcanzando   
1          1            2                  705.50    677.60  Alcanzando   
2          1            3                  753.02    535.10   Alcanzado   
3          1            4                 2078.33   1589.32  Alcanzando   
4          1            5                 1223.62    220.18  Alcanzando   

   VelNudos  VelNudosContacto Responsables  Contacto AccionContacto  \
0     10.97              8.99        Barco     Motor            N/C   
1     16.01             11.39        Barco  Pesquero            N/C   
2      1.00              5.40     Contacto    Velero             No   
3     23.31             13.51        Barco  Pesquero            N/C   
4     14.88             12.13        Barco     Motor            N/C   

        TCPA        Situacion_evaluacion       Responsable_evaluacion  \
0  97.930981                  Alcanzando                        Barco   
1   1.254746                  Alcanzando                        Barco   
2   3.564255  Alcanzado si no accion cto  Contacto que no toma accion   
3   4.031899                  Alcanzando                        Barco   
4  12.153389                  Alcanzando                        Barco   

  DificultadSituacion DificultadEjercicio  
0                BAJA               MEDIA  
1               MEDIA               MEDIA  
2                ALTA               MEDIA  
3               MEDIA               MEDIA  
4               MEDIA               MEDIA

In [14]:
# CELDA 4: Preprocesamiento de Datos de Entrenamiento y Definición de X, y

# Variables globales que se guardarán para usar en la predicción
columnas_categoricas_originales_para_dummies = []
columnas_X_finales = []
dificultad_map = {'BAJA': 0, 'MEDIA': 1, 'ALTA': 2} # Definición global

X_features = np.array([[]]) # Inicializar para evitar NameError
y_objetivo_modelo1 = np.array([]) # Inicializar

if df_datos.empty:
    print("El DataFrame df_datos (entrenamiento) está vacío. Saltando preprocesamiento.")
else:
    print("\n--- Iniciando Preprocesamiento de Datos de Entrenamiento ---")
    df_procesado = df_datos.copy()

    col_dificultad_situacion = 'DificultadSituacion'
    col_dificultad_ejercicio = 'DificultadEjercicio'

    for col_target in [col_dificultad_situacion, col_dificultad_ejercicio]:
        if col_target in df_procesado.columns:
            df_procesado[col_target] = df_procesado[col_target].map(dificultad_map)
    print(f"Columnas de dificultad mapeadas a números.")

    columnas_ids_y_etiquetas = ['Ejercicio', 'Situacion_n', col_dificultad_situacion, col_dificultad_ejercicio]
    columnas_features_potenciales = [col for col in df_procesado.columns if col not in columnas_ids_y_etiquetas]
    
    # Redefinimos aquí para asegurarnos que es del scope correcto
    columnas_categoricas_originales_para_dummies = [
        col for col in columnas_features_potenciales if df_procesado[col].dtype == 'object'
    ]
    print(f"Columnas categóricas originales (entrenamiento) para dummies: {columnas_categoricas_originales_para_dummies}")

    for col_cat in columnas_categoricas_originales_para_dummies:
        df_procesado[col_cat] = df_procesado[col_cat].replace('N/C', 'NC_Valor')
        moda_col = df_procesado[col_cat].mode()
        df_procesado[col_cat] = df_procesado[col_cat].fillna(moda_col[0] if not moda_col.empty else 'Desconocido')
    
    df_listo_para_xy = pd.get_dummies(df_procesado, 
                                      columns=columnas_categoricas_originales_para_dummies, 
                                      prefix=columnas_categoricas_originales_para_dummies, 
                                      dummy_na=False)
    print(f"Dimensiones (entrenamiento) después de get_dummies: {df_listo_para_xy.shape}")

    if col_dificultad_situacion in df_listo_para_xy.columns and not df_listo_para_xy[col_dificultad_situacion].isnull().all():
        y_objetivo_modelo1 = df_listo_para_xy[col_dificultad_situacion].values
    else:
        print(f"ERROR o ADVERTENCIA: '{col_dificultad_situacion}' no procesable en df_listo_para_xy.")
        y_objetivo_modelo1 = None # O np.array([])

    # Redefinimos aquí para asegurarnos que es del scope correcto
    columnas_X_finales = [col for col in df_listo_para_xy.columns if col not in columnas_ids_y_etiquetas]
    X_features = df_listo_para_xy[columnas_X_finales].values
    
    print(f"Matriz 'X_features' (entrenamiento) definida. Shape: {X_features.shape}")
    if y_objetivo_modelo1 is not None:
        print(f"Target 'y_objetivo_modelo1' ({col_dificultad_situacion}) definido. Shape: {y_objetivo_modelo1.shape}")

    try:
        with open('columnas_X_entrenamiento_final.pkl', 'wb') as f: pickle.dump(columnas_X_finales, f)
        print("Lista 'columnas_X_finales' guardada.")
        with open('columnas_categoricas_originales.pkl', 'wb') as f: pickle.dump(columnas_categoricas_originales_para_dummies, f)
        print("Lista 'columnas_categoricas_originales_para_dummies' guardada.")
    except Exception as e: print(f"Error guardando listas de columnas: {e}")
    print("--- Preprocesamiento de Entrenamiento Completado ---")


--- Iniciando Preprocesamiento de Datos de Entrenamiento ---
Columnas de dificultad mapeadas a números.
Columnas categóricas originales (entrenamiento) para dummies: ['Situacion', 'Responsables', 'Contacto', 'AccionContacto', 'Situacion_evaluacion', 'Responsable_evaluacion']
Dimensiones (entrenamiento) después de get_dummies: (1150, 40)
Matriz 'X_features' (entrenamiento) definida. Shape: (1150, 36)
Target 'y_objetivo_modelo1' (DificultadSituacion) definido. Shape: (1150,)
Lista 'columnas_X_finales' guardada.
Lista 'columnas_categoricas_originales_para_dummies' guardada.
--- Preprocesamiento de Entrenamiento Completado ---


In [15]:
# CELDA 5: División de Datos de Entrenamiento
print("\n--- Dividiendo Datos de Entrenamiento ---")
X_train, X_test, y_train, y_test = (np.array([]), np.array([]), np.array([]), np.array([]))

if X_features.size > 0 and y_objetivo_modelo1 is not None and y_objetivo_modelo1.size > 0 and \
   X_features.shape[0] == y_objetivo_modelo1.shape[0] and not pd.isna(y_objetivo_modelo1).all():
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_features, y_objetivo_modelo1, 
        test_size=0.25, 
        random_state=42, 
        stratify=y_objetivo_modelo1
    )
    print(f"Datos de entrenamiento divididos. X_train: {X_train.shape}, X_test: {X_test.shape}")
else:
    print("Error: X_features o y_objetivo_modelo1 no están listos para la división en el script de entrenamiento.")


--- Dividiendo Datos de Entrenamiento ---
Datos de entrenamiento divididos. X_train: (862, 36), X_test: (288, 36)


In [16]:
# CELDA 6: Modelo Básico SVM para DificultadSituacion (Entrenamiento)
print("\n--- Modelo SVM Básico para DificultadSituacion (Entrenamiento) ---")

if X_train.size > 0:
    # 🚀 Definimos GridSearchCV sobre un pipeline SVM polinómico
    svm_pipeline_basico = GridSearchCV(
        estimator=make_pipeline(
            StandardScaler(),
            SVC(kernel='poly', random_state=42, probability=True)
        ),
        param_grid={
            # Objetivo: maximizar accuracy probando un rango extenso de C y coef0
            'svc__C':       [1e-3, 1e-2, 1e-1, 1, 10, 100, 1e3],
            'svc__degree':  [2, 3, 4, 5],
            'svc__gamma':   ['scale', 'auto'],
            'svc__coef0':   [0.0, 0.1, 1.0, 10.0]
        },
        scoring='accuracy',   # KPI: accuracy
        cv=5,                 # StratifiedKFold interno
        n_jobs=-1,            # full throttle en GCP
        verbose=2
    )
    print(f"Pipeline SVM + GridSearch definido.")

    print("Entrenando svm_pipeline_basico (GridSearchCV)…")
    svm_pipeline_basico.fit(X_train, y_train)
    print("✅ GridSearchCV completado.")
    print("Mejores parámetros:", svm_pipeline_basico.best_params_)

    print("\nEvaluando en conjunto de prueba (entrenamiento)…")
    y_pred_test = svm_pipeline_basico.predict(X_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    print(f"Accuracy: {accuracy_test:.4f}")
    
    dificultad_map_inverso = {v: k for k, v in dificultad_map.items()}
    if y_train.size > 0 and not pd.isna(y_train).all():
        clases_unicas_ordenadas = sorted(np.unique(y_train[~pd.isna(y_train)]).astype(int))
        target_names_report = [dificultad_map_inverso.get(i, str(i)) for i in clases_unicas_ordenadas]
        print("\nReporte de Clasificación (Conjunto de Prueba del Entrenamiento):")
        print(classification_report(y_test, y_pred_test, target_names=target_names_report, labels=clases_unicas_ordenadas, zero_division=0))

        cm = confusion_matrix(y_test, y_pred_test, labels=clases_unicas_ordenadas)
        plt.figure(figsize=(6,4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=target_names_report, yticklabels=target_names_report)
        plt.xlabel('Predicción')
        plt.ylabel('Real')
        plt.title('Matriz de Confusión - SVM Básico (Entrenamiento)')
        plt.show()
    else:
        print("No se pudo generar el reporte de clasificación (y_train vacío o con NaNs).")

    nombre_modelo_guardado = 'modelo_svm_situacion_basico_entrenado.pkl'
    try:
        with open(nombre_modelo_guardado, 'wb') as file:
            pickle.dump(svm_pipeline_basico, file)
        print(f"\nModelo entrenado guardado como '{nombre_modelo_guardado}'.")
    except Exception as e:
        print(f"Error al guardar el modelo: {e}")
else:
    print("No se puede proceder con el entrenamiento del modelo, X_train está vacío.")



--- Modelo SVM Básico para DificultadSituacion (Entrenamiento) ---
Pipeline SVM + GridSearch definido.
Entrenando svm_pipeline_basico (GridSearchCV)…
Fitting 5 folds for each of 224 candidates, totalling 1120 fits


KeyboardInterrupt: 